In [1]:
import scanpy as sc
import scvi
import os
import seaborn as sns
import pandas as pd
import numpy as np


Global seed set to 0


In [2]:
# adata = sc.read_h5ad('/nfs_master/prakrithi/abhay/testis_allfiles/GSE112013_SRR6860519_Donor1_rep1.h5ad')

In [3]:
ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"


In [4]:
ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None)

In [5]:
def pp(data_path):
    adata = sc.read_h5ad(data_path)
    sc.pp.filter_genes(adata, min_cells = 10)
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df['prediction'] = solo.predict(soft = False)
    df.index = df.index.map(lambda x: x[:-2])
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
    
    adata = sc.read_h5ad(data_path)
    adata.obs['Sample'] = data_path.split('.')[0] #'raw_counts/GSM5226574_C51ctr_raw_counts.csv'
    
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]
    
    
    sc.pp.filter_cells(adata, min_genes=200) #get rid of cells with fewer than 200 genes
    #sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
    adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 20]
    adata = adata[adata.obs.pct_counts_ribo < 2]

    return adata

In [6]:
os.listdir('/nfs_master/prakrithi/abhay/testis_allfiles/')

['GSE154535_SRR12234213_Donor_OA.h5ad',
 'GSE153947_SRR12164935_Normal_3.h5ad',
 'GSE112013_SRR6860523_Donor3_rep1.h5ad',
 'GSE112013_SRR6860521_Donor2_rep1.h5ad',
 'GSE153947_SRR12164933_Normal_1.h5ad',
 'GSE153947_SRR12164938_Crypto_3.h5ad',
 'GSE154535_SRR12234211_Donor2_iNOA.h5ad',
 'GSE153947_SRR12164934_Normal_2.h5ad',
 'GSE112013_SRR6860524_Donor3_rep2.h5ad',
 'GSE154535_SRR12234212_Donor3_iNOA.h5ad',
 'GSE112013_SRR6860522_Donor2_rep2.h5ad',
 'GSE112013_SRR6860519_Donor1_rep1.h5ad',
 'GSE154535_SRR12234210_Donor1_iNOA.h5ad',
 'GSE112013_SRR6860520_Donor1_rep2.h5ad',
 'GSE153947_SRR12164936_Crypto_1.h5ad',
 'GSE153947_SRR12164937_Crypto_2.h5ad']

In [7]:
#will take 2-3 hours on HPC

path = '/nfs_master/prakrithi/abhay/testis_allfiles/'
out = []

for file in os.listdir(path):
    out.append(pp(path + file))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1937: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Epoch 400/400: 100%|████████████████████████████████████████████████████████████████| 400/400 [00:38<00:00, 10.48it/s, loss=527, v_num=1]
INFO     Creating doublets, preparing SOLO model.                                            


/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 131/400:  33%|████████████████████▎                                         | 131/400 [00:11<00:22, 11.75it/s, loss=0.468, v_num=1]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.485. Signaling Trainer to stop.


/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 400/400: 100%|████████████████████████████████████████████████████████████████| 400/400 [05:08<00:00,  1.30it/s, loss=357, v_num=1]
INFO     Creating doublets, preparing SOLO model.                                            


/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 351/400:  88%|██████████████████████████████████████████████████████▍       | 351/400 [03:16<00:27,  1.79it/s, loss=0.379, v_num=1]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.349. Signaling Trainer to stop.


/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


In [20]:
out[10]


View of AnnData object with n_obs × n_vars = 1010 × 58389
    obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [21]:
adata = sc.concat(out)

/home/prakrithi/miniconda3/envs/scTE2/lib/python3.7/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [22]:
adata

AnnData object with n_obs × n_vars = 12183 × 58389
    obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'

In [23]:
adata.obs

,Sample,doublet,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo
AAACGGGGTCTGGAGA-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,460,460,1021.0,0.0,0.0,12.0,1.175318
AACACGTGTCGAACAG-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,862,862,1407.0,0.0,0.0,25.0,1.776830
AACCGCGAGGAGTACC-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,840,840,2247.0,0.0,0.0,22.0,0.979083
AACCGCGCAGACAAAT-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,863,863,1367.0,0.0,0.0,22.0,1.609364
AACGTTGTCCTATGTT-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,386,386,733.0,0.0,0.0,10.0,1.364256
...,...,...,...,...,...,...,...,...,...
TTCTTAGGTGGAAAGA-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,1239,1239,4738.0,0.0,0.0,60.0,1.266357
TTGAACGGTAGCGCTC-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,432,432,810.0,0.0,0.0,14.0,1.728395
TTGGAACCATTGTGCA-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,591,591,1635.0,0.0,0.0,21.0,1.284404
TTGGCAAGTTCCAACA-1,/nfs_master/prakrithi/abhay/testis_allfiles/GS...,False,260,260,1184.0,0.0,0.0,18.0,1.520270
